In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import sklearn
from skimage.transform import resize

In [2]:
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [3]:
def resize_img(data):
    output = np.zeros((data.shape[0],64))
    for i, img in enumerate(data):
        output[i]=resize(img, (8,8)).flatten()
    return output

x_train_small = resize_img(x_train)
y_train_small = resize_img(y_train)
x_test_small = resize_img(x_test)
y_test_small = resize_img(y_test)

In [ ]:
# NORMALIZE!

In [ ]:
from sklearn.decomposition import PCA

In [8]:
from tensorflow.python.keras.models import load_model
import onnx
import keras2onnx

In [9]:
teacher_model = load_model('mnist8x8_100_200_300_200_100_10.h5')
student_model = load_model('mnist8x8_100_200_300_200_100_10-mirror.h5')

In [50]:
import pickle
properties = pickle.load(open("to_verify.pickle","rb"))

ModuleNotFoundError: No module named 'clustering'

In [ ]:
import copy
from keras.layers import Concatenate,Dense

def build_small_input_net(dim, x_train, x_test, y_test, teacher, student):
    pca = sklearn.decomposition.PCA(n_components=dim,whiten=True)
    pca.fit(x_train)
    print(f"Explained Variance: {np.sum(pca.explained_variance_ratio_)}")
    
    def weight_mat_init(shape, dtype=None):
        assert len(shape)==2 and (shape[0],shape[1])==pca.components_.shape, f"{shape},{pca.components_.shape}"
        return pca.components_
    
    layers = [
            keras.Input(shape=(dim,)),
            Dense(64, activation='linear', use_bias=False, trainable=False, kernel_initializer=weight_mat_init)
        ]
    x_test_small = pca.transform(x_test)
    print("Teacher...")
    teacher_layers = copy.copy(layers)
    for l in teacher.layers:
        teacher_layers.append(l)
    pca_teacher_model = keras.Sequential(
        teacher_layers,
        name="pca_teacher",
    )
    pca_teacher_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()])
    pca_teacher_model.summary()
    pca_teacher_model.evaluate(x_test_small, y_test)
    print("Student...")
    student_layers = copy.copy(layers)
    for l in student.layers:
        student_layers.append(l)
    pca_student_model = keras.Sequential(student_layers, name="pca_student")
    pca_student_model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.SparseCategoricalAccuracy()])
    pca_student_model.summary()
    pca_student_model.evaluate(x_test_small, y_test)

In [48]:
build_small_input_net(32, x_train_small, x_test_small, y_test, teacher_model, student_model)
    

Explained Variance: 0.9861906405603835
Teacher...
Model: "pca_teacher"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 64)                2048      
_________________________________________________________________
dense_45 (Dense)             (None, 100)               6500      
_________________________________________________________________
dense_46 (Dense)             (None, 200)               20200     
_________________________________________________________________
dense_47 (Dense)             (None, 300)               60300     
_________________________________________________________________
dense_48 (Dense)             (None, 200)               60200     
_________________________________________________________________
dense_49 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense